# Imitation Learning 모델 1
---
## Description
> SBA 5 조 스마트팩토리 자율주행을 위한 학습 프로그램. vanilla DAgger 모델. 

## todo
* [x] 모듈 1 구현
* [ ] 전처리 방법 다양화
* [x] 모듈 2 구현
* [ ] 모듈 3 구현
* [ ] pickle 등으로 로컬에 데이터 저장

## 링크
* [PEP8 코딩준수](https://kongdols-room.tistory.com/18)
* [openCV 가이드](https://opencv-python.readthedocs.io/en/latest/doc/02.videoStart/videoStart.html)
* [프로그레스 바 구현](https://wikidocs.net/13977)

## 1. 학습데이터 생성 모듈

In [1]:
import cv2
import numpy as np
import copy

In [2]:
'''class DataCollector:
    def __init__(self):
        pass'''
    

'class DataCollector:\n    def __init__(self):\n        pass'

In [3]:
class DataProducer:
    def __init__(self, width=320, height=240):
        self.observation = [] # 전처리된 영상 데이터 np 원본
        self.label = [] # 라벨링 데이터, observation 과 같은 크기
        self.width = width
        self.height = height
        print('> data producer 생성. ')
        print('> 가로 : {}, 세로 : {}'.format(width, height))
        print('-'*50)
        
    def __repr__(self):
        return '가로 : {}, 세로 : {}'.format(self.width, self.height)
    
    def produce_observation(self, file=None):
        # observation 을 생성하는 메서드
        
        if(file == None):
            # 재생할 원본영상이 없으면 실시간으로 카메라 영상 재생
            # 여기서는 노트북의 로컬카메라
            cap = cv2.VideoCapture(0)
            cap.set(3, self.width)
            cap.set(4, self.height)
        else:
            # 파일로부터 원본영상을 받아 재생
            # 처음부터 전처리한 영상을 안만드는 이유는
            # 하나의 원본으로부터 다양한 전처리 작업을 한 결과을 얻기 위함
            cap = cv2.VideoCapture(file)
        
        while(cap.isOpened()):
            ret, frame = cap.read()
            if(ret):
                gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
                ret_bin, binary = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY)
                #cv2.imshow('frame', gray)
                cv2.imshow('binary', binary)
                self.observation.append(binary)
                
                if cv2.waitKey(1) & 0xFF == ord('q'):
                    break
        cap.release()
        cv2.destroyAllWindows()
        print('> observation 데이터 생성완료, 원본 프레임 수 : {}'.format(len(self.observation)))
        print('-'*50)
    
    def produce_label(self, video, delay=10):
        # 외부에서 만든 영상파일을 받아 수동으로 라벨링을 하는 메서드
        # 수동으로 라벨링하는 과정을 observation 구하는 것과 분리함.
        # 다양한 라벨링을 테스트하기 위함
        # one_hot encoding 형태
        self.label = []
        command = [0,1,0]
        cap = cv2.VideoCapture(video)        
        while(cap.isOpened()):
            ret, frame = cap.read()
            if(ret):
                key = cv2.waitKey(delay) & 0xFF
                if(key == ord('q')):
                    break
                elif(key == ord('a')):
                    command = [1,0,0]
                elif(key == ord('d')):
                    command = [0,0,1]
                elif(key == ord('w')):
                    command = [0,1,0]
                
                self.label.append(command)
                cv2.putText(frame, 'command : {}'.format(command), (10, 30), cv2.FONT_HERSHEY_COMPLEX, .5, (0,0,255), 1)
                cv2.imshow('debug', frame)
            else:
                break
        cap.release()
        cv2.destroyAllWindows()
        print('> label 데이터 생성완료, label 수 : {}'.format(len(self.label)))
        print('-'*50)
    
    def get_observation(self):
        # 외부에서 영상을 만들기위해 원본을 제공
        video_data = copy.deepcopy(self.observation)
        print('> 전처리 영상 복사완료 - 프레임 수: {}'.format(len(self.observation)))
        print('-'*50)
        return video_data
    
    def get_label(self):
        # 라벨 데이터를 제공하는 메서드
        label_data = copy.deepcopy(self.label)
        print('> 라벨 데이터 복사완료 - 라벨 수: {}'.format(len(self.label)))
        return label_data
    
    def get_database(self):
        # observation - label 데이터 쌍을 data 학습데이터로 생성하는 메서드
        # CNN 에 사용하기 위해 4 dim 으로 reshape 한 후 최종 학습데이터 반환
        print('> 영상 프레임 수 : {}, 라벨 프레임 수 : {}, 부족한 라벨 수: {}'.format(len(self.observation), len(self.label), len(self.observation) - len(self.label)))
        print('> {} 프레임 학습 데이터 생성완료.'.format(len(self.label)))
        print('-'*50)
        selected_ob = self.observation[:len(self.label)]
        selected_ob = np.array(selected_ob)
        selected_ob = selected_ob.reshape([-1,240,320,1])
        return zip(selected_ob, self.label)

In [4]:
dp = DataProducer() # 학습데이터 생성기 객체 생성

> data producer 생성. 
> 가로 : 320, 세로 : 240
--------------------------------------------------


In [5]:
dp.produce_observation() # raw 영상으로부터 전처리영상 생성

> observation 데이터 생성완료, 원본 프레임 수 : 74
--------------------------------------------------


In [6]:
fourcc = cv2.VideoWriter_fourcc(*'MP42')
fps = 25.0
out = cv2.VideoWriter('output.avi', fourcc, fps, (320,240), 0) # 세밀한 컨트롤을 위해 프레임 늘리기 고려

In [7]:
# 전처리된 영상원본을 동영상파일로 저장
copied_observation = dp.get_observation()
for f in range(len(copied_observation)):
    frame = copied_observation[f]
    out.write(frame)
out.release()

> 전처리 영상 복사완료 - 프레임 수: 74
--------------------------------------------------


In [8]:
dp.produce_label('output.avi', delay=50) # 만들어진 동영상파일을 가지고 수동으로 라벨링하여 라벨을 프레임별로 따로 저장

> label 데이터 생성완료, label 수 : 74
--------------------------------------------------


In [9]:
data = list(dp.get_database()) # observation - label 모음 학습데이터를 획득한다.
for i, d in enumerate(data):
    #shaped = np.reshape(d[0],[-1,240,320,1])
    pass

> 영상 프레임 수 : 74, 라벨 프레임 수 : 74, 부족한 라벨 수: 0
> 74 프레임 학습 데이터 생성완료.
--------------------------------------------------


## 2. 신경망 학습 모듈

### tensorflow 관련
* feed 데이터는 기본 자료형, numpy 가능. tensor 는 순수한 값이 아닌 연산노드이므로 계산불가
* `squeeze()`, `expand_dim()` 으로 dimension 조절
* one-hot 인코딩 데이터 배열을 비교할 때는 `np.all(a==b)`
* logit 과 sigmoid, softmax 의 관계는 역함수 관계.
* `softmax_cross_entropy_with_logits_v2` 는 네트워크의 출력인 logit 을 그대로 때려박아서 분류결과를 내놓는다.
* softmax(logit) 하고 corssentropy 를 적용하는 작업을 합친것임. logit 은 softmax 의 역함수이므로
---
### TODO
* [ ] 네트워크 변형하며 성능 테스트
* [ ] label 데이터의 형태 변형하여 테스트 - 현재는 3 개 카테고리의 분류문제인데 너무 단순하고 practical 하지 않음
* [ ] 실제로는 steering angle 에 대응하는 floating number 로 regression output 이 필요
* [ ] 검증하는 코드 작성하기
* [ ] CPU 점유율 너무높음. AWS 쓰게 해줘 ㅠㅠ
* Imitation Learning 을 위해 output 은 더 세밀한 action 이 필요

In [10]:
import tensorflow as tf

In [11]:
class CNN:
    def __init__(self, h, w, sess):
        self.size_h = h
        self.size_w = w
        self.sess = sess
        self.model = self.make_model()
    
    def make_model(self):
        # 모델
        self.observation = tf.placeholder(shape=[None, self.size_h, self.size_w, 1], dtype=tf.float32) # 이미지 데이터
        self.label = tf.placeholder(shape=[None, 3], dtype=tf.int16) # 라벨 데이터
        
        self.w_in = tf.Variable(tf.random_normal([5,5,1,8], stddev=.01)) # conv 1 가중치
        self.l1 = tf.nn.conv2d(self.observation, self.w_in, strides=[1,1,1,1], padding='SAME')
        self.l1 = tf.nn.relu(self.l1)
        self.l1 = tf.nn.max_pool(self.l1, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')
        
        self.w_out = tf.Variable(tf.random_normal(shape=[self.size_w//2*self.size_h//2*8, 3], stddev=.01))
        self.b = tf.Variable(tf.random_normal([3]))
        self.h_flat = tf.reshape(self.l1, [-1, self.size_w//2*self.size_h//2*8])
        
        self.output = tf.matmul(self.h_flat, self.w_out) + self.b
        self.cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=self.output, labels=self.label))
        self.optimizer = tf.train.AdamOptimizer(learning_rate=.001).minimize(self.cost)
        print('> 모델 생성 완료')
        
    def train(self, batch_in, batch_label):
        # 학습시행.
        _, cost = self.sess.run([self.optimizer, self.cost], feed_dict={self.observation:batch_in, self.label:batch_label})
        return cost
    
    def test(self, d, y):
        # 길을 잘 인식하는지 테스트.
        # 테스트 데이터 셋을 미리 저장해놓고 검증하는 메서드
        # 시각화 필요
        out = self.sess.run(self.output, feed_dict={self.observation:d})
        idx = np.argmax(out, axis=1)
        one_hot = np.zeros_like(out)
        cnt = 0
        for i, arr in enumerate(one_hot):
            arr[idx[i]] = 1
            if(np.all(arr == y[i])):
                cnt += 1
        
        accuracy = cnt / len(d)
        return accuracy
    
    def test_live(self):
        # 실시간으로 길을 잘 인식하는지 테스트하는 메서드.
        out = self.sess.run(self.output, feed_dict={self.observation:d})


In [15]:
# 하이퍼 파라미터
learning_rate = .001
total_epoch = 20

# train data, test data 절반씩 나누기
batch_in_train = [data[i][0] for i in range(len(data)//2)]
batch_label_train = [data[i][1] for i in range(len(data)//2)]
batch_in_test = [data[i][0] for i in range(len(data)//2, len(data))]
batch_label_test = [data[i][1] for i in range(len(data)//2, len(data))]

print('> 총 데이터 갯수 : {}, 학습 데이터 갯수 : {}, 검증 데이터 갯수 : {}'.format(len(data), len(batch_in_train), len(batch_in_test)))
init = tf.global_variables_initializer()

with tf.Session() as sess:
    print('start')
    model = CNN(240, 320, sess)
    init = tf.global_variables_initializer()
    sess.run(init)
    for epoch in range(total_epoch):
        cost = model.train(batch_in_train, batch_label_train)
        print('{} epoch, cost : {}'.format(epoch+1, cost))

> 총 데이터 갯수 : 74, 학습 데이터 갯수 : 37, 검증 데이터 갯수 : 37
start
> 모델 생성 완료
1 epoch, cost : 26.682661056518555
2 epoch, cost : 366.2832946777344
3 epoch, cost : 183.99240112304688
4 epoch, cost : 34.15765380859375
5 epoch, cost : 0.3319648504257202
6 epoch, cost : 11.845327377319336
7 epoch, cost : 4.226559638977051
8 epoch, cost : 0.5367381572723389
9 epoch, cost : 0.043232642114162445
10 epoch, cost : 0.019332807511091232
11 epoch, cost : 0.07625644654035568
12 epoch, cost : 0.120908722281456
13 epoch, cost : 0.1279595047235489
14 epoch, cost : 0.12072278559207916
15 epoch, cost : 0.1053871214389801
16 epoch, cost : 0.08260931819677353
17 epoch, cost : 0.058390360325574875
18 epoch, cost : 0.03774141147732735
19 epoch, cost : 0.02301478385925293
20 epoch, cost : 0.013968897983431816


## 3. 실시간 Data Aggregatioin 모듈